In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Change de répertoire vers ton projet
%cd /content/drive/MyDrive/PROJET_IA/PROJET_IA/donnees_dossier/Besoin_Client_3

/content/drive/MyDrive/PROJET_IA/PROJET_IA/donnees_dossier/Besoin_Client_3


In [ ]:
!pip install --upgrade pip            # Met à jour pip (optionnel)
!pip install -r requirements.txt      # Installe pandas, scikit-learn, joblib…

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 29.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [ ]:
# Par exemple, au début de train_model.py :
import os
from pathlib import Path

# On peut forcer la base des données ici
# BASE_DIR = Path("/content/drive/MyDrive/PROJET_IA/...")
# mais normalement le parser argparse s’en charge.

# Vérifie que l’usage est bien :
# python scripts/train_model.py --data data/export_IA.csv --models-dir models scripts/train_model.py

In [ ]:
!python scripts/train_model.py --csv data/export_IA.csv --models-dir models --max-samples 50000


2025-06-12 09:20:49.955873: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749720049.991579    4139 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749720050.001969    4139 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
[INFO] GPU: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
[INFO] 413487 lignes, 149 MMSI

>> Horizon 5 min
Train 50000 | Test 72034
I0000 00:00:1749720084.168393    4139 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
Epoch 1/20
I0000 00:00:1749720087.018654    4306 cuda_dnn.cc:52

In [ ]:
!python scripts/predict_position.py --input data/export_IA.csv --models-dir models --output preds.csv


2025-06-12 09:42:23.871965: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749721343.893629   10561 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749721343.900320   10561 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
[INFO] Données chargées: 413487 lignes, 149 navires
[INFO] 410656 séquences créées pour la prédiction

[INFO] Prédiction pour horizon 5 minutes...
2025-06-12 09:45:50.345521: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
I0000 00:00:1749721550.345736   10561 gpu_device.cc:2022] Created device /

In [ ]:
import pandas as pd

# Load the predictions from the CSV file
preds = pd.read_csv('preds.csv')

# Load the true data from the CSV file, specifying dtypes to handle mixed types
true_df = pd.read_csv('data/export_IA.csv', dtype={'VesselName': str, 'IMO': str, 'CallSign': str, 'VesselType': str, 'Status': str})


# 1. Créer une colonne "Date_Pred" dans preds = BaseDateTime + Horizon_min minutes
preds['BaseDateTime'] = pd.to_datetime(preds['BaseDateTime']) # Convert BaseDateTime to datetime
preds['Date_Pred'] = preds['BaseDateTime'] + pd.to_timedelta(preds['Horizon_min'], unit='m')

# Convert BaseDateTime in true_df to datetime
true_df['BaseDateTime'] = pd.to_datetime(true_df['BaseDateTime'])

# Renommer les colonnes LAT et LON pour les vraies données in true_df
true_df = true_df.rename(columns={"LAT": "True_LAT", "LON": "True_LON"})


# 2. Trier les dataframes par MMSI et date (nécessaire pour merge_asof dans chaque groupe)
preds = preds.sort_values(['MMSI', 'Date_Pred']).reset_index(drop=True)
true_df = true_df.sort_values(['MMSI', 'BaseDateTime']).reset_index(drop=True)

# Group by MMSI and perform merge_asof for each group
merged_list = []
for mmsi, preds_group in preds.groupby("MMSI"):
    true_group = true_df[true_df["MMSI"] == mmsi].copy()

    if not true_group.empty:
        merged_group = pd.merge_asof(
            preds_group,
            true_group[['BaseDateTime', 'True_LAT', 'True_LON']],
            left_on='Date_Pred',
            right_on='BaseDateTime',
            direction='nearest',  # nearest c’est mieux que 'backward' pour plus de précision
            tolerance=pd.Timedelta('5min')  # tolérance max, à ajuster si besoin
        )
        merged_list.append(merged_group)

# Concatenate the results
merged = pd.concat(merged_list, ignore_index=True)


# Display the head of the merged DataFrame to check the result
print("Head of the merged DataFrame:")
display(merged.head())

<ipython-input-3-1020030862>:7: DtypeWarning: Columns (13,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  true_df = pd.read_csv('data/export_IA.csv', dtype={'VesselName': str, 'IMO': str, 'CallSign': str, 'VesselType': str, 'Status': str})


Head of the merged DataFrame:


,MMSI,BaseDateTime_x,Horizon_min,Predicted_LAT,Predicted_LON,Sequence_ID,Date_Pred,BaseDateTime_y,True_LAT,True_LON
0,205776000,2023-05-25 01:37:27+00:00,5,26.050587,-97.024150,0,2023-05-25 01:42:27+00:00,2023-05-25 01:43:27+00:00,25.95845,-97.37878
1,205776000,2023-05-25 01:37:27+00:00,10,26.008354,-96.827034,0,2023-05-25 01:47:27+00:00,2023-05-25 01:46:27+00:00,25.95845,-97.37881
2,205776000,2023-05-25 01:43:27+00:00,5,26.050007,-97.023020,1,2023-05-25 01:48:27+00:00,2023-05-25 01:49:27+00:00,25.95845,-97.37877
3,205776000,2023-05-25 01:46:27+00:00,5,26.049610,-97.022620,2,2023-05-25 01:51:27+00:00,2023-05-25 01:52:27+00:00,25.95843,-97.37878
4,205776000,2023-05-25 01:37:27+00:00,15,25.999456,-97.153190,0,2023-05-25 01:52:27+00:00,2023-05-25 01:52:27+00:00,25.95843,-97.37878


In [ ]:
import folium
from datetime import timedelta
from math import radians, sin, cos, sqrt, atan2

# Fonction haversine pour calculer la distance entre deux points GPS
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # rayon de la Terre en km
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c  # distance en km

# Forcer le type datetime et trier
preds["BaseDateTime"] = pd.to_datetime(preds["BaseDateTime"])
true_df["BaseDateTime"] = pd.to_datetime(true_df["BaseDateTime"])
preds = preds.sort_values("BaseDateTime")
true_df = true_df.sort_values("BaseDateTime")

# Filtrer uniquement les prédictions à 5 min
preds_5min = preds[preds["Horizon_min"] == 5]

# Limiter à 10 bateaux
mmsi_list = preds_5min["MMSI"].unique()[:50]

# Calcul du centre de la carte
initial_points = []
for mmsi in mmsi_list:
    true_mmsi = true_df[true_df["MMSI"] == mmsi]
    preds_mmsi = preds_5min[preds_5min["MMSI"] == mmsi]
    if true_mmsi.empty or preds_mmsi.empty:
        continue
    initial_time = preds_mmsi.iloc[0]["BaseDateTime"]
    init_real_point = true_mmsi[true_mmsi["BaseDateTime"] == initial_time]
    if not init_real_point.empty:
        initial_points.append((
            init_real_point["True_LAT"].iloc[0],
            init_real_point["True_LON"].iloc[0]
        ))

if initial_points:
    avg_lat = sum([p[0] for p in initial_points]) / len(initial_points)
    avg_lon = sum([p[1] for p in initial_points]) / len(initial_points)
else:
    first_pred = preds_5min.iloc[0]
    avg_lat, avg_lon = first_pred["Predicted_LAT"], first_pred["Predicted_LON"]

# Créer la carte centrée
m = folium.Map(location=[avg_lat, avg_lon], zoom_start=8)

# Boucle sur chaque bateau limité à 10
for mmsi in mmsi_list:
    preds_mmsi = preds_5min[preds_5min["MMSI"] == mmsi].copy()
    true_mmsi = true_df[true_df["MMSI"] == mmsi].copy()

    if preds_mmsi.empty or true_mmsi.empty:
        continue

    row = preds_mmsi.iloc[0]
    initial_time = row["BaseDateTime"]
    pred_time = initial_time + timedelta(minutes=int(row["Horizon_min"]))

    pred_lat = row["Predicted_LAT"]
    pred_lon = row["Predicted_LON"]

    window = timedelta(minutes=2)
    nearby = true_mmsi[
        (true_mmsi["BaseDateTime"] >= pred_time - window) &
        (true_mmsi["BaseDateTime"] <= pred_time + window)
    ]

    if not nearby.empty:
        real_lat = nearby["True_LAT"].mean()
        real_lon = nearby["True_LON"].mean()
    else:
        real_lat = None
        real_lon = None

    initial_real_point = true_mmsi[true_mmsi["BaseDateTime"] == initial_time]

    # Marqueur point initial réel
    if not initial_real_point.empty:
        folium.Marker(
            location=[initial_real_point["True_LAT"].iloc[0], initial_real_point["True_LON"].iloc[0]],
            popup=f"Bateau {mmsi} - Initial Real",
            icon=folium.Icon(color="green", icon="ship", prefix='fa')
        ).add_to(m)

    # Marqueur prédiction
    folium.Marker(
        location=[pred_lat, pred_lon],
        popup=f"Bateau {mmsi} - Prediction +5min",
        icon=folium.Icon(color="blue", icon="hourglass-half", prefix='fa')
    ).add_to(m)

    # Marqueur position réelle correspondante
    if real_lat is not None and real_lon is not None:
        folium.Marker(
            location=[real_lat, real_lon],
            popup=f"Bateau {mmsi} - True +5min (approximated)",
            icon=folium.Icon(color="red", icon="map-marker", prefix='fa')
        ).add_to(m)

        # Calcul distance et ajout d'une ligne entre les 2 points
        dist_km = haversine(pred_lat, pred_lon, real_lat, real_lon)

        # Ligne entre la position prédite et réelle
        folium.PolyLine(
            locations=[(pred_lat, pred_lon), (real_lat, real_lon)],
            color="purple",
            weight=2,
            opacity=0.7,
            popup=f"Distance: {dist_km:.2f} km"
        ).add_to(m)

        # Popup distance sur la ligne (il faut contourner car PolyLine n'a pas de popup natif)
        midpoint = [(pred_lat + real_lat) / 2, (pred_lon + real_lon) / 2]
        folium.Marker(
            location=midpoint,
            icon=folium.DivIcon(
                html=f'<div style="font-size:12px; color:purple; font-weight:bold;">{dist_km:.2f} km</div>'
            )
        ).add_to(m)

# Sauvegarder et afficher la carte
m.save("map_10_bateaux_distance.html")
print("Carte sauvegardée : map_10_bateaux_distance.html")

display(m)

Carte sauvegardée : map_10_bateaux_distance.html
